In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from datetime import datetime

Covid Ages Data from https://data.ca.gov/dataset/covid-19-cases/resource/926fd08f-cc91-4828https://data.ca.gov/dataset/covid-19-cases/resource/926fd08f-cc91-4828-af38-bd45de97f8c3-af38-bd45de97f8c3

In [2]:
# read covid age demographics
covid_ages = pd.read_csv('data/case_demographics_age.csv')
covid_ages.head()

# lots of missing data at beginning -> perhaps because nothing to compare it to

,age_group,totalpositive,date,case_percent,deaths,deaths_percent,ca_percent
0,0-17,120,2020-04-02,NaN,NaN,NaN,NaN
1,18-49,5302,2020-04-02,NaN,NaN,NaN,NaN
2,50-64,2879,2020-04-02,NaN,NaN,NaN,NaN
3,65 and Older,2342,2020-04-02,NaN,NaN,NaN,NaN
4,Unknown,58,2020-04-02,NaN,NaN,NaN,NaN


In [3]:
covid_ages.info()
#1235 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age_group       1235 non-null   object 
 1   totalpositive   1235 non-null   int64  
 2   date            1235 non-null   object 
 3   case_percent    1135 non-null   float64
 4   deaths          1135 non-null   float64
 5   deaths_percent  1135 non-null   float64
 6   ca_percent      1125 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 67.7+ KB


In [4]:
covid_ages['age_group'].unique()

array(['0-17', '18-49', '50-64', '65 and Older', 'Unknown', '65+',
       'Missing'], dtype=object)

Covid Data from https://data.ca.gov/dataset/covid-19-cases/resource/926fd08f-cc91-4828-af38-bd45de97f8c3

In [5]:
covid_cases = pd.read_csv('data/statewide_cases.csv')
covid_cases.head()

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,2020-03-18
1,Santa Clara,183.0,8.0,32,2,2020-03-19
2,Santa Clara,246.0,8.0,63,0,2020-03-20
3,Santa Clara,269.0,10.0,23,2,2020-03-21
4,Santa Clara,284.0,13.0,15,3,2020-03-22


In [6]:
covid_cases.info()
#15665 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15665 entries, 0 to 15664
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               15665 non-null  object 
 1   totalcountconfirmed  15662 non-null  float64
 2   totalcountdeaths     15663 non-null  float64
 3   newcountconfirmed    15665 non-null  int64  
 4   newcountdeaths       15665 non-null  int64  
 5   date                 15665 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 734.4+ KB


Notes about what is observed from the data:

- Both have the same Date Format, which is YYYY/MM/DD Time, so we can potentially join on date
- The covid_cases is sorted by date and grouped by county, but the covid_ages doesn't show us the counties. We would need to sum totalcountconfirmed in covid_cases grouped by dates before joining and comparing the proportions; We will also probably need to do that for the age groups...
- Unknown and missing age group in covid_ages; what are they? How should we account for them?

In [7]:
grouped_covid_cases = covid_cases.groupby('date')['totalcountconfirmed'].sum()
grouped_covid_cases

date
2020-03-18        675.0
2020-03-19       1006.0
2020-03-20       1224.0
2020-03-21       1468.0
2020-03-22       1733.0
                ...    
2020-11-30    1225189.0
2020-12-01    1245948.0
2020-12-02    1264539.0
2020-12-03    1286557.0
2020-12-04    1311625.0
Name: totalcountconfirmed, Length: 262, dtype: float64

In [8]:
grouped_covid_ages = covid_ages.groupby(['date','age_group'])['totalpositive'].sum()
grouped_covid_ages

date        age_group   
2020-04-02  0-17               120
            18-49             5302
            50-64             2879
            65 and Older      2342
            Unknown             58
                             ...  
2020-12-04  0-17            147807
            18-49           780365
            50-64           246393
            65+             135981
            Missing           1079
Name: totalpositive, Length: 1235, dtype: int64

In [9]:
# change grouped series to dataframe and reset index to get
# age group and date out of index
grouped_covid_ages = grouped_covid_ages.to_frame()
grouped_covid_ages = grouped_covid_ages.reset_index()
grouped_covid_cases = grouped_covid_cases.to_frame()
grouped_covid_cases = grouped_covid_cases.reset_index()

In [10]:
grouped_covid_ages

,date,age_group,totalpositive
0,2020-04-02,0-17,120
1,2020-04-02,18-49,5302
2,2020-04-02,50-64,2879
3,2020-04-02,65 and Older,2342
4,2020-04-02,Unknown,58
...,...,...,...
1230,2020-12-04,0-17,147807
1231,2020-12-04,18-49,780365
1232,2020-12-04,50-64,246393
1233,2020-12-04,65+,135981


In [11]:
grouped_covid_cases

,date,totalcountconfirmed
0,2020-03-18,675.0
1,2020-03-19,1006.0
2,2020-03-20,1224.0
3,2020-03-21,1468.0
4,2020-03-22,1733.0
...,...,...
257,2020-11-30,1225189.0
258,2020-12-01,1245948.0
259,2020-12-02,1264539.0
260,2020-12-03,1286557.0


In [13]:
print(grouped_covid_ages[grouped_covid_ages['age_group'] == '65 and Older'])
print(grouped_covid_ages[grouped_covid_ages['age_group'] == '65+'])
# 65 and Older changed to 65+ after 4/28/2020
# Rename 65+ to 65 and Older
grouped_covid_ages = grouped_covid_ages.replace('65+', '65 and Older')


           date     age_group  totalpositive
3    2020-04-02  65 and Older           2342
8    2020-04-03  65 and Older           2660
13   2020-04-04  65 and Older           2966
18   2020-04-05  65 and Older           3129
23   2020-04-06  65 and Older           3516
28   2020-04-07  65 and Older           3751
33   2020-04-08  65 and Older           4056
38   2020-04-09  65 and Older           4335
43   2020-04-10  65 and Older           4561
48   2020-04-11  65 and Older           4870
53   2020-04-12  65 and Older           5036
58   2020-04-13  65 and Older           5320
63   2020-04-14  65 and Older           5605
68   2020-04-15  65 and Older           6051
73   2020-04-16  65 and Older           6360
78   2020-04-17  65 and Older           6729
83   2020-04-18  65 and Older           7071
88   2020-04-19  65 and Older           7267
93   2020-04-20  65 and Older           7650
98   2020-04-21  65 and Older           8035
103  2020-04-22  65 and Older           8401
108  2020-

In [14]:
# Taking out the missing data (Unknown and Missing age groups)
# Unknown age group renamed to Missing after 6/01/2020
print(grouped_covid_ages[grouped_covid_ages['age_group'] == 'Unknown'])
print(grouped_covid_ages[grouped_covid_ages['age_group'] == 'Missing'])
# Replace Unknown with Missing age group name
grouped_covid_ages = grouped_covid_ages.replace('Unknown', 'Missing')


           date age_group  totalpositive
4    2020-04-02   Unknown             58
9    2020-04-03   Unknown             59
14   2020-04-04   Unknown             49
19   2020-04-05   Unknown             51
24   2020-04-06   Unknown             55
29   2020-04-07   Unknown             56
34   2020-04-08   Unknown             59
39   2020-04-09   Unknown             52
44   2020-04-10   Unknown             49
49   2020-04-11   Unknown             54
54   2020-04-12   Unknown             58
59   2020-04-13   Unknown             62
64   2020-04-14   Unknown             69
69   2020-04-15   Unknown             79
74   2020-04-16   Unknown             64
79   2020-04-17   Unknown             67
84   2020-04-18   Unknown             72
89   2020-04-19   Unknown             63
94   2020-04-20   Unknown             66
99   2020-04-21   Unknown             67
104  2020-04-22   Unknown             84
109  2020-04-23   Unknown             81
114  2020-04-24   Unknown             88
119  2020-04-25 

In [15]:
grouped_covid_ages

,date,age_group,totalpositive
0,2020-04-02,0-17,120
1,2020-04-02,18-49,5302
2,2020-04-02,50-64,2879
3,2020-04-02,65 and Older,2342
4,2020-04-02,Missing,58
...,...,...,...
1230,2020-12-04,0-17,147807
1231,2020-12-04,18-49,780365
1232,2020-12-04,50-64,246393
1233,2020-12-04,65 and Older,135981


In [16]:
# Delete missing data from total


In [17]:
# join the dataframes
covid_df = pd.merge(grouped_covid_ages, grouped_covid_cases)
covid_df

,date,age_group,totalpositive,totalcountconfirmed
0,2020-04-02,0-17,120,10701.0
1,2020-04-02,18-49,5302,10701.0
2,2020-04-02,50-64,2879,10701.0
3,2020-04-02,65 and Older,2342,10701.0
4,2020-04-02,Missing,58,10701.0
...,...,...,...,...
1230,2020-12-04,0-17,147807,1311625.0
1231,2020-12-04,18-49,780365,1311625.0
1232,2020-12-04,50-64,246393,1311625.0
1233,2020-12-04,65 and Older,135981,1311625.0


In [18]:
covid_df.head()

,date,age_group,totalpositive,totalcountconfirmed
0,2020-04-02,0-17,120,10701.0
1,2020-04-02,18-49,5302,10701.0
2,2020-04-02,50-64,2879,10701.0
3,2020-04-02,65 and Older,2342,10701.0
4,2020-04-02,Missing,58,10701.0


In [19]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235 entries, 0 to 1234
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 1235 non-null   object 
 1   age_group            1235 non-null   object 
 2   totalpositive        1235 non-null   int64  
 3   totalcountconfirmed  1235 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 48.2+ KB


Where to go from here:
- Pair with demographic age group size
- Reverse engineer how many people in each age group tested +
- Which age group contracted Covid/is at higher risk for contracting it?
- Proportion of new cases vs population size

CA population (2019) data from https://censusreporter.org/data/table/?table=B01001&primary_geo_id=04000US06&geo_ids=04000US06,01000US

In [20]:
population = pd.read_csv('data/CA_population2019.csv')

In [21]:
# truncate dataframe
ca_population = population[2:20]
ca_population

,Label,California!!Total!!Estimate,California!!Total!!Margin of Error,California!!Percent!!Estimate,California!!Percent!!Margin of Error,California!!Male!!Estimate,California!!Male!!Margin of Error,California!!Percent Male!!Estimate,California!!Percent Male!!Margin of Error,California!!Female!!Estimate,California!!Female!!Margin of Error,California!!Percent Female!!Estimate,California!!Percent Female!!Margin of Error
2,Under 5 years,"2,377,839","±3,612",6.0%,±0.1,"1,215,418","±3,802",6.2%,±0.1,"1,162,421","±3,349",5.8%,±0.1
3,5 to 9 years,"2,380,762","±21,917",6.0%,±0.1,"1,209,648","±17,120",6.2%,±0.1,"1,171,114","±14,583",5.9%,±0.1
4,10 to 14 years,"2,629,510","±20,779",6.7%,±0.1,"1,350,641","±16,386",6.9%,±0.1,"1,278,869","±14,246",6.4%,±0.1
5,15 to 19 years,"2,547,449","±7,689",6.4%,±0.1,"1,295,491","±5,128",6.6%,±0.1,"1,251,958","±5,829",6.3%,±0.1
6,20 to 24 years,"2,638,791","±6,800",6.7%,±0.1,"1,360,772","±5,142",6.9%,±0.1,"1,278,019","±4,272",6.4%,±0.1
7,25 to 29 years,"3,085,132","±5,462",7.8%,±0.1,"1,596,197","±4,445",8.1%,±0.1,"1,488,935","±3,731",7.5%,±0.1
8,30 to 34 years,"2,950,920","±5,779",7.5%,±0.1,"1,520,141","±4,080",7.7%,±0.1,"1,430,779","±3,848",7.2%,±0.1
9,35 to 39 years,"2,764,917","±25,008",7.0%,±0.1,"1,412,165","±17,207",7.2%,±0.1,"1,352,752","±13,954",6.8%,±0.1
10,40 to 44 years,"2,533,994","±24,316",6.4%,±0.1,"1,270,506","±16,867",6.5%,±0.1,"1,263,488","±14,212",6.4%,±0.1
11,45 to 49 years,"2,512,971","±4,451",6.4%,±0.1,"1,248,279","±3,373",6.4%,±0.1,"1,264,692","±3,126",6.4%,±0.1


In [22]:
# Rename Columns
columns = {'Label': 'Label', 
           'California!!Total!!Estimate': 'Total Estimate', 
           'California!!Total!!Margin of Error': 'Total Margin of Error',
           'California!!Percent!!Estimate': 'Percent Estimate', 
           'California!!Percent!!Margin of Error': 'Percent Margin of Error',
           'California!!Male!!Estimate': 'Male Estimate',
           'California!!Male!!Margin of Error': 'Male Margin of Error',
           'California!!Percent Male!!Estimate': 'Percent Male Estimate',
           'California!!Percent Male!!Margin of Error': 'Percent Male Margin of Error',
           'California!!Female!!Estimate': 'Female Estimate',
           'California!!Female!!Margin of Error': 'Female Margin of Error',
           'California!!Percent Female!!Estimate': 'Percent Female Estimate', 
           'California!!Percent Female!!Margin of Error': 'Percent Female Margin of Error'
          }
ca_population = ca_population.rename(columns = columns)

In [23]:
# Change Total Estimate, Male Estimate, and Female Estimate columnns as ints
change = ['Total Estimate', 'Male Estimate', 'Female Estimate']

# take out commas from the numbers and cast as int64
for col in change:
    ca_population[col] = ca_population[col].str.replace(',', '').astype('int64')


In [24]:
ca_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 2 to 19
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Label                           18 non-null     object
 1   Total Estimate                  18 non-null     int64 
 2   Total Margin of Error           18 non-null     object
 3   Percent Estimate                18 non-null     object
 4   Percent Margin of Error         18 non-null     object
 5   Male Estimate                   18 non-null     int64 
 6   Male Margin of Error            18 non-null     object
 7   Percent Male Estimate           18 non-null     object
 8   Percent Male Margin of Error    18 non-null     object
 9   Female Estimate                 18 non-null     int64 
 10  Female Margin of Error          18 non-null     object
 11  Percent Female Estimate         18 non-null     object
 12  Percent Female Margin of Error  18 non-null     obje

In [25]:
ca_population

,Label,Total Estimate,Total Margin of Error,Percent Estimate,Percent Margin of Error,Male Estimate,Male Margin of Error,Percent Male Estimate,Percent Male Margin of Error,Female Estimate,Female Margin of Error,Percent Female Estimate,Percent Female Margin of Error
2,Under 5 years,2377839,"±3,612",6.0%,±0.1,1215418,"±3,802",6.2%,±0.1,1162421,"±3,349",5.8%,±0.1
3,5 to 9 years,2380762,"±21,917",6.0%,±0.1,1209648,"±17,120",6.2%,±0.1,1171114,"±14,583",5.9%,±0.1
4,10 to 14 years,2629510,"±20,779",6.7%,±0.1,1350641,"±16,386",6.9%,±0.1,1278869,"±14,246",6.4%,±0.1
5,15 to 19 years,2547449,"±7,689",6.4%,±0.1,1295491,"±5,128",6.6%,±0.1,1251958,"±5,829",6.3%,±0.1
6,20 to 24 years,2638791,"±6,800",6.7%,±0.1,1360772,"±5,142",6.9%,±0.1,1278019,"±4,272",6.4%,±0.1
7,25 to 29 years,3085132,"±5,462",7.8%,±0.1,1596197,"±4,445",8.1%,±0.1,1488935,"±3,731",7.5%,±0.1
8,30 to 34 years,2950920,"±5,779",7.5%,±0.1,1520141,"±4,080",7.7%,±0.1,1430779,"±3,848",7.2%,±0.1
9,35 to 39 years,2764917,"±25,008",7.0%,±0.1,1412165,"±17,207",7.2%,±0.1,1352752,"±13,954",6.8%,±0.1
10,40 to 44 years,2533994,"±24,316",6.4%,±0.1,1270506,"±16,867",6.5%,±0.1,1263488,"±14,212",6.4%,±0.1
11,45 to 49 years,2512971,"±4,451",6.4%,±0.1,1248279,"±3,373",6.4%,±0.1,1264692,"±3,126",6.4%,±0.1


In [26]:
# Merge the age groups to reflect the age groups in Covid data
# Age groups for Covid: 0-17, 18-49, 50-64, 65 and Older, Unknown, Missing
until_19 = ca_population[0:4]['Total Estimate'].sum()
from_20_49 = ca_population[4:10]['Total Estimate'].sum()
from_50_64 = ca_population[10:13]['Total Estimate'].sum()
older_65 = ca_population[13:]['Total Estimate'].sum()
print(until_19, from_20_49, from_50_64, older_65)

9935560 16486725 7254940 5834998


For next time:
- Create new DF with the age groups from population
- Age groups are slightly different (0-17 in Covid vs 0-19 in Population)
- Need to take out data for missing and unknown age groups
- Make proportions columnns

Under 17 age group data frfom: https://www.kidsdata.org/topic/34/child-population-age-gender/table#fmt=141&loc=2&tf=110&ch=1433,926,927,1434,1435,372,78,77,79&sortColumnId=0&sortType=asc

In [27]:
# Subtract ages 18-19 from the until_19 age group
# read in table of California age group of people 0-17
children_df = pd.read_csv('data/CA_Under17Population.csv')
children_df

,FIPSCode,LocationType,Location,AgeGroup,Gender,TimeFrame,DataFormat,Data
0,6,State,California,Ages 0_2,Female,2020,Number,"666,477"
1,6,State,California,Ages 0_2,Male,2020,Number,"697,215"
2,6,State,California,Ages 0_2,Total,2020,Number,"1,363,692"
3,6,State,California,Ages 11_13,Female,2020,Number,"770,448"
4,6,State,California,Ages 11_13,Male,2020,Number,"800,496"
5,6,State,California,Ages 11_13,Total,2020,Number,"1,570,944"
6,6,State,California,Ages 14_17,Female,2020,Number,"1,017,713"
7,6,State,California,Ages 14_17,Male,2020,Number,"1,055,961"
8,6,State,California,Ages 14_17,Total,2020,Number,"2,073,674"
9,6,State,California,Ages 3_5,Female,2020,Number,"715,327"


In [28]:
# Extract necessary row and delete all columns that aren't the total
children = children_df.iloc[-1][-1]
type(children) # str
# Convert to int64
chidren_no_commas = children.replace(',', '')
under_17 = int(chidren_no_commas)

In [29]:
# Subtract under_17 from until_19
ages18_19 = until_19 - under_17
# add ages18_19 to from_20_49
from_18_49 = sum([ages18_19, from_20_49])
print(f'until 19: {until_19}\n\
0-17: {under_17}\n\
18-19: {ages18_19}\n\
20-49: {from_20_49}\n\
18-49: {from_18_49}\n\
50-64: {from_50_64}\n\
65 and older: {older_65}')

until 19: 9935560
0-17: 9026052
18-19: 909508
20-49: 16486725
18-49: 17396233
50-64: 7254940
65 and older: 5834998


In [30]:
# Create DataFrame of the age group populations
# ['0-17', '18-49', '50-64', '65 and Older', 'Unknown', '65+', 'Missing']
d = {'0-17': under_17, '18-49': from_18_49,\
        '50-64': from_50_64, '65 and Older': older_65}
ages = pd.Series(data=d)
ages

0-17             9026052
18-49           17396233
50-64            7254940
65 and Older     5834998
dtype: int64

In [31]:
# Create proportions of age_group positive rate vs age_group population
pop_proportions = []
for pos, age in zip(covid_df['totalpositive'], covid_df['age_group']):
    if age == '0-17':
        pop_proportions.append(round(pos/ages[0], 5))
    elif age == '18-49':
        pop_proportions.append(round(pos/ages[1], 5))
    elif age == '50-64':
        pop_proportions.append(round(pos/ages[2], 5))
    elif age == '65 and Older':
        pop_proportions.append(round(pos/ages[3], 5))
    else:
        pop_proportions.append(0)

In [32]:
covid_df['agegroup_proportion'] = pop_proportions

In [33]:
covid_df

,date,age_group,totalpositive,totalcountconfirmed,agegroup_proportion
0,2020-04-02,0-17,120,10701.0,0.00001
1,2020-04-02,18-49,5302,10701.0,0.00030
2,2020-04-02,50-64,2879,10701.0,0.00040
3,2020-04-02,65 and Older,2342,10701.0,0.00040
4,2020-04-02,Missing,58,10701.0,0.00000
...,...,...,...,...,...
1230,2020-12-04,0-17,147807,1311625.0,0.01638
1231,2020-12-04,18-49,780365,1311625.0,0.04486
1232,2020-12-04,50-64,246393,1311625.0,0.03396
1233,2020-12-04,65 and Older,135981,1311625.0,0.02330


In [34]:
grouped_covid_cases

,date,totalcountconfirmed
0,2020-03-18,675.0
1,2020-03-19,1006.0
2,2020-03-20,1224.0
3,2020-03-21,1468.0
4,2020-03-22,1733.0
...,...,...
257,2020-11-30,1225189.0
258,2020-12-01,1245948.0
259,2020-12-02,1264539.0
260,2020-12-03,1286557.0
